In [364]:
import json
import graphviz
import copy

In [365]:
with open('../testcases/test4/NFA.json') as f:
    nfa = json.load(f)


terminating_states = set()
#loop through the states, and make each transition a list if it isn't already
for state in nfa:
    if state == 'startingState':
        nfa[state] = [nfa[state]]
        continue
    
    for transition in nfa[state]:
        if transition == 'isTerminatingState':
            if nfa[state][transition] == True:
                terminating_states.add(state)
            continue
        if type(nfa[state][transition]) != list:
            nfa[state][transition] = [nfa[state][transition]]

print(nfa)
print(terminating_states)
starting_state_origin = nfa['startingState']

{'startingState': ['S3'], 'S3': {'isTerminatingState': False, 'epsilon': ['S1', 'S4']}, 'S4': {'isTerminatingState': False, 'epsilon': ['S7']}, 'S1': {'isTerminatingState': False, 'a': ['S2']}, 'S2': {'isTerminatingState': False, 'epsilon': ['S3', 'S4']}, 'S7': {'isTerminatingState': False, 'epsilon': ['S5', 'S8']}, 'S8': {'isTerminatingState': False, 'epsilon': ['S15']}, 'S5': {'isTerminatingState': False, 'b': ['S6']}, 'S6': {'isTerminatingState': False, 'epsilon': ['S7', 'S8']}, 'S15': {'isTerminatingState': False, 'epsilon': ['S13', 'S16']}, 'S16': {'isTerminatingState': True}, 'S13': {'isTerminatingState': False, 'epsilon': ['S11', 'S9']}, 'S14': {'isTerminatingState': False, 'epsilon': ['S15', 'S16']}, 'S11': {'isTerminatingState': False, 'b': ['S12']}, 'S12': {'isTerminatingState': False, 'epsilon': ['S14']}, 'S9': {'isTerminatingState': False, 'a': ['S10']}, 'S10': {'isTerminatingState': False, 'epsilon': ['S14']}}
{'S16'}


## NFA 2 DFA

In [366]:
#global variable to store the closures of each state
#used to avoid recalculating the closure of a state
#DP approach
global_closuers = dict()

def get_possible_inputs(states):
    possible_transitions = set()
    # print(states)
    for state in states:
        for symbol in nfa[state]:
            if symbol != "epsilon" and symbol != "isTerminatingState":
                # print(symbol)
                possible_transitions.add(symbol)
    return possible_transitions

def epsilon_closure(nfa, state, prev_closures):
    # print(state)
    if state in global_closuers:
        return global_closuers[state]

    #initialize the closure of the current state
    # with the current state
    closure = set()
    closure.add(state)
    prev_closures.add(state)
    # add epsilon transitions of the current state
    epsilon_transitions = set(nfa[state].get("epsilon", []))
    
    # print(epsilon_transitions)
    for s in epsilon_transitions:
        #union of the current closure and the closure of the next state
        if s not in prev_closures:
            closure |= epsilon_closure(nfa, s, prev_closures)

    global_closuers[state] = closure
    return closure

def move(nfa, states, symbol):
    result = set()
    for state in states:
        if symbol in nfa[state]:
            if symbol == "epsilon" or symbol == "isTerminatingState":
                continue
            #print(nfa[state][symbol])
            result |= set(nfa[state][symbol])
    return result

def nfa_to_dfa(nfa):
    dfa = {}
    #used to use as a key in dfa, sets are not hashable
    #because they are mutable, but frozensets are immutable
    starting_state = nfa["startingState"][0]
    start_state = frozenset(epsilon_closure(nfa, starting_state, set()))
    is_terminating = False

    dfa["startingState"] = start_state
    dfa[start_state] = {}
    queue = [start_state]
    while queue:
        current_state = queue.pop(0)
        if current_state & terminating_states:
            is_terminating = True
        dfa[current_state]["isTerminatingState"] = is_terminating
            
        possible_inputs = get_possible_inputs(current_state)
        for minput in possible_inputs:
            possible_transitions = move(nfa, current_state, minput)
            next_states_closures = set()
            for next_state in possible_transitions:
                next_states_closures |= epsilon_closure(nfa, next_state, set())
                
            closure = frozenset(next_states_closures)
            if closure not in dfa:
                dfa[closure] = {}
                queue.append(closure)
            dfa[current_state][minput] = closure
            
    return dfa

dfa = nfa_to_dfa(nfa)
print(dfa)

{'startingState': frozenset({'S8', 'S4', 'S11', 'S16', 'S13', 'S7', 'S9', 'S1', 'S5', 'S15', 'S3'}), frozenset({'S8', 'S4', 'S11', 'S16', 'S13', 'S7', 'S9', 'S1', 'S5', 'S15', 'S3'}): {'isTerminatingState': True, 'a': frozenset({'S2', 'S8', 'S4', 'S11', 'S10', 'S16', 'S13', 'S7', 'S9', 'S1', 'S14', 'S5', 'S15', 'S3'}), 'b': frozenset({'S8', 'S11', 'S16', 'S13', 'S7', 'S9', 'S14', 'S5', 'S12', 'S6', 'S15'})}, frozenset({'S2', 'S8', 'S4', 'S11', 'S10', 'S16', 'S13', 'S7', 'S9', 'S1', 'S14', 'S5', 'S15', 'S3'}): {'isTerminatingState': True, 'a': frozenset({'S2', 'S8', 'S4', 'S11', 'S10', 'S16', 'S13', 'S7', 'S9', 'S1', 'S14', 'S5', 'S15', 'S3'}), 'b': frozenset({'S8', 'S11', 'S16', 'S13', 'S7', 'S9', 'S14', 'S5', 'S12', 'S6', 'S15'})}, frozenset({'S8', 'S11', 'S16', 'S13', 'S7', 'S9', 'S14', 'S5', 'S12', 'S6', 'S15'}): {'isTerminatingState': True, 'a': frozenset({'S14', 'S16', 'S11', 'S10', 'S13', 'S15', 'S9'}), 'b': frozenset({'S8', 'S11', 'S16', 'S13', 'S7', 'S9', 'S14', 'S5', 'S12', 'S

## Clean States

In [367]:
def clean_DFA(dfa):
    index = 1
    dfa_cleaned = {}
    mapping = {}

    mapping[dfa["startingState"]] = "S0"

    for state in dfa:
        
        if state == "startingState":
            continue

        if mapping.get(state) == None:      
            state_name = "S" + str(index)
            mapping[state] = state_name
            index += 1


    for state in dfa:
        if state == "startingState":
            dfa_cleaned[state] = mapping[dfa[state]]
            continue
        
        dfa_cleaned[mapping[state]] = {"isTerminatingState": dfa[state]["isTerminatingState"]}

        for transition in dfa[state]:
            if transition == "isTerminatingState":
                continue

            dfa_cleaned[mapping[state]][transition] = {mapping[dfa[state][transition]]}

    return dfa_cleaned

dfa_cleaned = clean_DFA(dfa)
starting_state_origin = dfa_cleaned["startingState"]
print(dfa_cleaned)

{'startingState': 'S0', 'S0': {'isTerminatingState': True, 'a': {'S1'}, 'b': {'S2'}}, 'S1': {'isTerminatingState': True, 'a': {'S1'}, 'b': {'S2'}}, 'S2': {'isTerminatingState': True, 'a': {'S3'}, 'b': {'S2'}}, 'S3': {'isTerminatingState': True, 'a': {'S3'}, 'b': {'S4'}}, 'S4': {'isTerminatingState': True, 'a': {'S3'}, 'b': {'S4'}}}


## Draw DFA

In [368]:
def draw_dfa(dfa, name='dfa_graph'):
    dot = graphviz.Digraph()
    starting_state = dfa['startingState']
    for state, transitions in dfa.items():
        if state == 'startingState':
            continue 
        if state == starting_state:
            dot.node(state, shape='doublecircle' if transitions['isTerminatingState'] else 'circle', color='blue')
        else:
            dot.node(state, shape='doublecircle' if transitions['isTerminatingState'] else 'circle')

        for symbol, next_states in transitions.items():
            if symbol != 'isTerminatingState' and symbol != 'startingState':
                for next_state in next_states:
                    dot.edge(state, next_state, label=symbol)

    dot.render(name, format='png', cleanup=True)

In [369]:
draw_dfa(dfa_cleaned, 'dfa_graph1')

## Minimize DFA

In [370]:
#at the start, partition of 1 is the set of all terminating states
#and partition 2 is the set of all non-terminating states
def create_partitions(dfa):
    partitions = {1: set(), 2: set()}
    for state in dfa:
        if state == "startingState":
            continue
        if dfa[state]["isTerminatingState"]:
            partitions[1].add(state)
        else:
            partitions[2].add(state)

    #if one of them is empty, just remove it
    if not partitions[1]:
        partitions.pop(1)

    if not partitions[2]:
        partitions.pop(2)
        
    return partitions

def minimize_dfa(dfa, partitions):
    new_partitions = copy.deepcopy(partitions)
    while True:
        try:
            for partition in partitions:
                
                list_splited_partitions = []

                if len(partitions[partition]) == 1:
                    continue

                pivot_state = next(iter(partitions[partition]))

                for state in partitions[partition]:
                    if pivot_state == state:
                        continue

                    if not are_equivalent(dfa, pivot_state, state, new_partitions[partition]):
                        new_partitions[partition].remove(state)

                        for splited_partition in list_splited_partitions:
                            temp_state = next(iter(new_partitions[splited_partition]))
                            if are_equivalent(dfa, state, temp_state, new_partitions[splited_partition]):
                                new_partitions[splited_partition].add(state)
                                break
                        else:   
                            list_splited_partitions.append(len(new_partitions)+1)
                            new_partitions[len(new_partitions) + 1] = {state}
                                     
                if new_partitions == partitions:
                    raise 

                partitions = copy.deepcopy(new_partitions)
        except Exception as e:
            break

    return partitions

def are_equivalent(dfa, state1, state2, partition):
    for symbol in dfa[state1]:
        if symbol == "isTerminatingState":
            continue

        if dfa[state2].get(symbol) == None:
            return False

        if (dfa[state1][symbol] != dfa[state2][symbol]) \
            and not (next(iter(dfa[state1][symbol])) in partition and next(iter(dfa[state2][ symbol])) in partition):
            return False
   
            
    return True                

def merge_partitions(dfa, partitions):
    # merged_dfa = {"startingState": partitions[1].pop()}
    merged_dfa = {}
    pivots = {}
    dfa = copy.deepcopy(dfa)
    
    for key, partition in partitions.items():
        pivot_state = next(iter(partition))
        merged_transitions = {"isTerminatingState": dfa[pivot_state]["isTerminatingState"]}
        pivots[key] = pivot_state

        for state in partition:
            if state == starting_state_origin:
                merged_dfa["startingState"] = pivot_state

            merged_transitions |= dfa[state]

        merged_dfa[pivot_state] = merged_transitions

    #loop over all transitions, and if the transition contains any state
    #that belongs to a partition, change it to the pivot state
    for state in merged_dfa:
        if state == "startingState":
            if not merged_dfa[state] in pivots.values():
                pivot_pratition = find_partition(partitions, merged_dfa[state])
                merged_dfa[state] = pivots[pivot_pratition]
            continue

        for symbol in merged_dfa[state]:
            if symbol == "isTerminatingState":
                continue

            if not next(iter(merged_dfa[state][symbol])) in pivots.values():
                pivot_partition = find_partition(partitions, next(iter(merged_dfa[state][symbol])))
                merged_dfa[state][symbol].discard(next(iter(merged_dfa[state][symbol])))
                merged_dfa[state][symbol].add(pivots[pivot_partition])
           
    return merged_dfa

                
def find_partition(partitions, state):
    for partition in partitions:
        if state in partitions[partition]:
            return partition

    return None

print(dfa_cleaned)
partitions = create_partitions(dfa_cleaned)
print(partitions)
minimized_dfa = minimize_dfa(dfa_cleaned, partitions)
print(minimized_dfa)
combined_dfa = merge_partitions(dfa_cleaned, minimized_dfa)
print(combined_dfa)


{'startingState': 'S0', 'S0': {'isTerminatingState': True, 'a': {'S1'}, 'b': {'S2'}}, 'S1': {'isTerminatingState': True, 'a': {'S1'}, 'b': {'S2'}}, 'S2': {'isTerminatingState': True, 'a': {'S3'}, 'b': {'S2'}}, 'S3': {'isTerminatingState': True, 'a': {'S3'}, 'b': {'S4'}}, 'S4': {'isTerminatingState': True, 'a': {'S3'}, 'b': {'S4'}}}
{1: {'S2', 'S4', 'S1', 'S0', 'S3'}}
{1: {'S2', 'S4', 'S1', 'S0', 'S3'}}
{'startingState': 'S2', 'S2': {'isTerminatingState': True, 'a': {'S2'}, 'b': {'S2'}}}


In [371]:
draw_dfa(combined_dfa, 'dfa_graph2')